In [1]:
%load_ext autoreload
%autoreload 2

Here I use the DOW as the MMI and the S&P 500 as the CAPMMI

1

In [2]:
import numpy as np
from utils.pandas import to_portfolio_ts
from utils.pandas import beta

In [3]:
from data.dow import load

mmi = load(start_date='2015-01-01')[['Date', 'Close', 'ticker']].rename(columns={'Date': 'date', 'Close': 'price'})
mmi = mmi.set_index(['date', 'ticker']).sort_index()

In [4]:
N = mmi.index.get_level_values(level='ticker').nunique()
N

30

In [5]:
mmi['lag_price'] = mmi.groupby(level='ticker')['price'].shift(1)
mmi['ret'] = mmi['price']/mmi['lag_price'] - 1
mmi = mmi.dropna()

In [6]:
vols = mmi.groupby(level='ticker')['ret'].std()
avg_vol = vols.mean()
avg_vol

np.float64(0.01698887369524248)

In [7]:
eqp = to_portfolio_ts(mmi.copy())
port_vol = eqp.std()
port_vol

np.float64(0.011014660681230637)

In [8]:
avg_corr = ((port_vol/avg_vol)**2 * N - 1) / (N-1)
avg_corr

np.float64(0.4003640901780653)

2

In [9]:
from data.sp500 import load as load_sp500

capmmi = load_sp500(start_date='2015-01-01')[['Date', 'Close', 'ticker']].rename(columns={'Date': 'date', 'Close': 'price'})
capmmi = capmmi.set_index(['date', 'ticker']).sort_index()
capmmi['lag_price'] = capmmi.groupby(level='ticker')['price'].shift(1)
capmmi['ret'] = capmmi['price']/capmmi['lag_price'] - 1
capmmi = capmmi.dropna()

Since the index composition changed in the time period. Removing the stocks that were added or removed in the period from the index

In [10]:
all_tcks = capmmi.groupby(level='ticker').apply(lambda x: x.index.get_level_values(level='date').nunique())
tcks = all_tcks[(all_tcks >= all_tcks.median())].rename('ndates')

In [11]:
capmmi = capmmi.join(tcks, how='inner').drop(columns=['ndates'])
capmmi.index.get_level_values(level='ticker').nunique()

465

In [12]:
benchmark = to_portfolio_ts(capmmi.copy())
benchmark = benchmark.rename('market_ret')

In [13]:
mmi = mmi.join(benchmark, how='inner')

In [14]:
betas = mmi.groupby(level='ticker').apply(lambda x: beta(x['ret'], x['market_ret'])[0])

In [15]:
avg_beta = betas.mean()
avg_beta

np.float64(0.9102542945980464)

In [16]:
benchmark_risk = benchmark.std()
benchmark_risk

np.float64(0.011639885252285213)

In [17]:
# total risk of each asset is its standard deviation
vols

ticker
AAPL    0.018320
AMGN    0.015731
AMZN    0.020765
AXP     0.019658
BA      0.025449
CAT     0.018832
CRM     0.021932
CSCO    0.015773
CVX     0.018604
DIS     0.017798
GS      0.018437
HD      0.015329
HON     0.014536
IBM     0.015381
JNJ     0.011544
JPM     0.017266
KO      0.011250
MCD     0.012856
MMM     0.016085
MRK     0.013998
MSFT    0.017122
NKE     0.019470
NVDA    0.031009
PG      0.011719
SHW     0.016514
TRV     0.015180
UNH     0.017829
V       0.015413
VZ      0.012327
WMT     0.013540
Name: ret, dtype: float64

In [18]:
risks = vols.rename('total_risk').to_frame().join(betas.rename('beta'))
risks['residual_risk'] = risks['total_risk'] - risks['beta']*benchmark_risk

In [19]:
risks.mean().rename('mean')

total_risk       0.016989
beta             0.910254
residual_risk    0.006394
Name: mean, dtype: float64

3.

In [20]:
eqp = eqp.rename('port_ret').to_frame()
eqp = eqp.join(benchmark, how='inner')

In [21]:
eqp['tracking_error'] = eqp['port_ret'] - eqp['market_ret']
active_risk = eqp['tracking_error'].std()
active_risk

np.float64(0.003206379293818309)

In [22]:
t1 = (benchmark_risk*(risks['beta'].mean()-1))**2
t2 = risks['residual_risk'].mean()**2 / N
np.sqrt(t1+t2)

np.float64(0.0015664810906919216)

True active risk was more than double of what would be estimated by making equal average vol and uncorrelated vol assumptions

Let's increase the portfolio size by selecting random 100 stocks from S&P 500

In [23]:
del mmi

In [24]:
np.random.seed(11)
N = 100
keys = np.random.choice(capmmi.index.get_level_values(level='ticker').unique(), size=N, replace=False)

In [25]:
mmi2 = capmmi[capmmi.index.get_level_values(level='ticker').isin(keys)]
mmi2 = mmi2.join(benchmark, how='inner')

In [26]:
vols2 = mmi2.groupby(level='ticker')['ret'].std()
eqp2 = to_portfolio_ts(mmi2.copy())

In [27]:
betas2 = mmi2.groupby(level='ticker').apply(lambda x: beta(x['ret'], x['market_ret'])[0])
risks2 = vols2.rename('total_risk').to_frame().join(betas2.rename('beta'))
risks2['residual_risk'] = risks2['total_risk'] - risks2['beta']*benchmark_risk
risks2.mean().rename('mean')

total_risk       0.019844
beta             1.001888
residual_risk    0.008182
Name: mean, dtype: float64

In [28]:
eqp2 = eqp2.rename('port_ret').to_frame()
eqp2 = eqp2.join(benchmark, how='inner')
eqp2['tracking_error'] = eqp2['port_ret'] - eqp2['market_ret']
active_risk = eqp2['tracking_error'].std()
active_risk

np.float64(0.0014668897723248266)

In [29]:
t1 = (benchmark_risk*(risks2['beta'].mean()-1))**2
t2 = risks2['residual_risk'].mean()**2 / N
np.sqrt(t1+t2)

np.float64(0.0008184736086841451)